# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-54f5b7e1cb-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Production company explorative search") 

Consider the following exploratory information need:

> investigate the main companies working on cinema-related content. We want to know which are the main television production companies and the film production companies, which company distributes more film and some consideration about awards.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P106`     | profession   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q36479`    | The Lion King      | node |





Also consider

```
wd:Q36479 ?p ?obj .
```

is the BGP to retrieve all **properties of The Lion King**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for films

2. Identify the BGP for Netflix

3. Identify the BGP for television production company

4. Identify the BGP for film production company

5. Find the top-5 companies (amongst television production company and film production company) which produced the highest number of crime films (the result set must be a list of triples company IRI, label and #crime film).

6. Find the company (amongst television production company and film production company) which distributed more films (of any kind of film genre) that they did not produce (the result set must be a list of triples company IRI, label and #film).

7. Find how many company are listed in every stock exchange market (the result set must be a list of triples stock exchange IRI, label and #companies). 

8. Identify the BGP for Academy Award

9. Find the companies (among television and film production companies) that won at least 5 Academy Awards for Best Actress for the movies they produced (the result set must be a list of triples company IRI, label and #awards).

## Task 1
Identify the BGP for films

In [2]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q36479 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3092'), ('name', 'film crew member')]
[('p', 'http://www.wikidata.org/prop/direct/P4021'), ('name', 'danskefilm animated film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7679'), ('name', 'Cinépolis KLIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('name', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('name', 'Box Office Mojo film ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('name', 'AlloCiné film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia B

In [3]:
# Lion King all props
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q36479 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
ORDER BY ASC(?name)
LIMIT 1000
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3593'), ('name', 'AFI Catalog of Feature Films ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7118'), ('name', 'AMPAS collections film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P6145'), ('name', 'Academy Awards Database film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('name', 'AllMovie title ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2465'), ('name', 'Allcinema film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('name', 'AlloCiné film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9586'), ('name', 'Apple TV movie ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3280'), ('name', 'BAnQ author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P4632'), ('name', 'Bechdel Test Movie List ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5384'), ('name', 'Behind The Voice Actors film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P950'), ('name', 'Biblioteca Nacional de España ID')]

In [4]:
# Lion King instance of
queryString = """
SELECT DISTINCT ?obj ?oName
WHERE {
   # bind something
   wd:Q36479 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?oName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q29168811'), ('oName', 'animated feature film')]
1


In [5]:
# animated feature film
queryString = """
SELECT DISTINCT ?p ?name ?obj ?oName
WHERE {
   # bind something
   wd:Q29168811 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?oName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P461'), ('name', 'opposite of'), ('obj', 'http://www.wikidata.org/entity/Q17517379'), ('oName', 'animated short film')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q202866'), ('oName', 'animated film')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q24869'), ('oName', 'feature film')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q8254220'), ('oName', 'Category:Animated feature films')]
4


In [6]:
# animated film
queryString = """
SELECT DISTINCT ?p ?name ?obj ?oName
WHERE {
   # bind something
   wd:Q202866 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?oName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P345'), ('oName', 'IMDb ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5869'), ('name', 'model item'), ('obj', 'http://www.wikidata.org/entity/Q134430'), ('oName', 'Snow White and the Seven Dwarfs')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P2047'), ('oName', 'duration')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q6522809'), ('oName', 'Category:Animated films')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P2079'), ('oName', 'fabrication method')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P921'), ('oNa

In [7]:
# feature film
queryString = """
SELECT DISTINCT ?p ?name ?obj ?oName
WHERE {
   # bind something
   wd:Q24869 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?oName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q11424'), ('oName', 'film')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('name', 'has quality'), ('obj', 'http://www.wikidata.org/entity/Q2199864'), ('oName', 'duration')]
[('p', 'http://www.wikidata.org/prop/direct/P461'), ('name', 'opposite of'), ('obj', 'http://www.wikidata.org/entity/Q24862'), ('oName', 'short film')]
[('p', 'http://www.wikidata.org/prop/direct/P5135'), ('name', 'greater than'), ('obj', 'http://www.wikidata.org/entity/Q24862'), ('oName', 'short film')]
[('p', 'http://www.wikidata.org/prop/direct/P5135'), ('name', 'greater than'), ('obj', 'http://www.wikidata.org/entity/Q24865'), ('oName', 'medium-length film')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category"), ('obj', 'http://www.wikidata.org/entity/Q32118748'), ('oName', 'Category:Feature films')]
6


Final query for this task

In [8]:
# BGP for film
queryString = """
SELECT *
WHERE {
   BIND(wd:Q11424 AS ?iri)
   ?iri sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('iri', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
1


## Task 2
Identify the BGP for Netflix

In [9]:
#props of netflix id of Lion King
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q36479 wdt:P1874 ?obj . #obj is netflx id
   ?obj ?p ?o.
   ?p sc:name ?name.
}
ORDER BY ASC(?name)
LIMIT 1000
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [10]:
#what distributed Lion King
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   wd:Q36479 wdt:P750 ?obj . #distributed by obj
   ?obj  sc:name ?name.
}
ORDER BY ASC(?name)
LIMIT 1000
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q54958752'), ('name', 'Disney+')]
[('obj', 'http://www.wikidata.org/entity/Q1097348'), ('name', 'InterCom')]
[('obj', 'http://www.wikidata.org/entity/Q135288'), ('name', 'Microsoft Store')]
[('obj', 'http://www.wikidata.org/entity/Q907311'), ('name', 'Netflix')]
[('obj', 'http://www.wikidata.org/entity/Q1323594'), ('name', 'Walt Disney Studios Motion Pictures')]
5


Final query for this task

In [11]:
# BGP for Netflix
queryString = """
SELECT *
WHERE {
   BIND(wd:Q907311 AS ?iri)
   ?iri sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('iri', 'http://www.wikidata.org/entity/Q907311'), ('name', 'Netflix')]
1


## Task 3
Identify the BGP for television production company

In [12]:
# Netflix props
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q907311 ?p ?o.
   ?p sc:name ?name.
}
LIMIT 1000
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P8939'), ('name', 'Google Play developer slug')]
[('p', 'http://www.wikidata.org/prop/direct/P1056'), ('name', 'product or material produced')]
[('p', 'http://www.wikidata.org/prop/direct/P112'), ('name', 'founded by')]
[('p', 'http://www.wikidata.org/prop/direct/P1128'), ('name', 'employees')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1454'), ('name', 'legal form')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P1581'), ('name', 'official blog')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters lo

In [13]:
# Netflix instance of
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q907311 wdt:P31 ?o.
   ?o sc:name ?name.
}
LIMIT 1000
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q10689397'), ('name', 'television production company')]
[('o', 'http://www.wikidata.org/entity/Q4830453'), ('name', 'business')]
[('o', 'http://www.wikidata.org/entity/Q723685'), ('name', 'video on demand')]
[('o', 'http://www.wikidata.org/entity/Q891723'), ('name', 'public company')]
[('o', 'http://www.wikidata.org/entity/Q15265344'), ('name', 'broadcaster')]
[('o', 'http://www.wikidata.org/entity/Q368290'), ('name', 'film distributor')]
[('o', 'http://www.wikidata.org/entity/Q841645'), ('name', 'Internet television')]
[('o', 'http://www.wikidata.org/entity/Q1194970'), ('name', 'dot-com company')]
8


Final query for this task

In [14]:
# BGP for television production company
queryString = """
SELECT *
WHERE {
   BIND(wd:Q10689397 AS ?iri)
   ?iri sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('iri', 'http://www.wikidata.org/entity/Q10689397'), ('name', 'television production company')]
1


## Task 4
Identify the BGP for film production company

In [15]:
# tv prod company props
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q10689397 ?p ?o.
   ?p sc:name ?name.
}
LIMIT 1000
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P452'), ('name', 'industry')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
7


In [16]:
# tv prod company subclass of
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q10689397 wdt:P279 ?o.
   ?o sc:name ?name.
}
LIMIT 1000
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1762059'), ('name', 'film production company')]
[('o', 'http://www.wikidata.org/entity/Q19364326'), ('name', 'media studio facility')]
[('o', 'http://www.wikidata.org/entity/Q11396960'), ('name', 'production company')]
3


Final query for this task

In [17]:
# BGP for film production company
queryString = """
SELECT *
WHERE {
   BIND(wd:Q1762059 AS ?iri)
   ?iri sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('iri', 'http://www.wikidata.org/entity/Q1762059'), ('name', 'film production company')]
1


## Task 5
Find the top-5 companies (amongst television production company and film production company) which produced the highest number of crime films (the result set must be a list of triples company IRI, label and #crime film).

In [18]:
# Lion King production company
queryString = """
SELECT DISTINCT ?obj ?oName
WHERE {
   # bind something
   wd:Q36479 wdt:P272 ?obj .
   # get the label
   ?obj sc:name ?oName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q191224'), ('oName', 'Walt Disney Pictures')]
[('obj', 'http://www.wikidata.org/entity/Q1047410'), ('oName', 'Walt Disney Animation Studios')]
2


In [19]:
# Lion King producer
queryString = """
SELECT DISTINCT ?obj ?oName
WHERE {
   # bind something
   wd:Q36479 wdt:P162 ?obj .
   # get the label
   ?obj sc:name ?oName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1072843'), ('oName', 'Don Hahn')]
1


In [20]:
# instances of film
queryString = """
SELECT DISTINCT ?f ?oName
WHERE {
   # bind something
   ?f wdt:P31 wd:Q11424.
   # get the label
   ?f sc:name ?oName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('f', 'http://www.wikidata.org/entity/Q8178307'), ('oName', '15 Park Avenue')]
[('f', 'http://www.wikidata.org/entity/Q8197668'), ('oName', 'Idiot Love')]
[('f', 'http://www.wikidata.org/entity/Q319263'), ('oName', 'Doctor in the House')]
[('f', 'http://www.wikidata.org/entity/Q8452990'), ('oName', "Sunday League - Pepik Hnatek's Final Match")]
[('f', 'http://www.wikidata.org/entity/Q284184'), ('oName', 'Ra.One')]
[('f', 'http://www.wikidata.org/entity/Q13637926'), ('oName', 'Indra')]
[('f', 'http://www.wikidata.org/entity/Q13638153'), ('oName', 'Inner Senses')]
[('f', 'http://www.wikidata.org/entity/Q13645807'), ('oName', 'Madhumasam')]
[('f', 'http://www.wikidata.org/entity/Q13647258'), ('oName', 'Marupakkam')]
[('f', 'http://www.wikidata.org/entity/Q13677566'), ('oName', 'Timberjack')]
[('f', 'http://www.wikidata.org/entity/Q13693967'), ('oName', 'Cross')]
[('f', 'http://www.wikidata.org/entity/Q15464247'), ('oName', 'What the Fish')]
[('f', 'http://www.wikidata.org/entity/

In [21]:
# props of random film (Little Man, What Now?)
queryString = """
SELECT ?p ?name ?o ?oName
WHERE {
   wd:Q15859668 ?p ?o.
   ?p sc:name ?name.
   ?o sc:name ?oName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('name', 'based on'), ('o', 'http://www.wikidata.org/entity/Q1746652'), ('oName', 'Little Man, What Now?')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q91333'), ('oName', 'Jakob Tiedtke')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q90022'), ('oName', 'Blandine Ebinger')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q11424'), ('oName', 'film')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre'), ('o', 'http://www.wikidata.org/entity/Q130232'), ('oName', 'drama')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('name', 'country of origin'), ('o', 'http://www.wikidata.org/entity/Q183'), ('oName', 'Germany')]
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('name', 'original language of film o

In [22]:
#drama instance of
queryString = """
SELECT DISTINCT ?o ?oName
WHERE {
    wd:Q130232 wdt:P31 ?o.
   ?o sc:name ?oName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q201658'), ('oName', 'film genre')]
1


In [23]:
#all instances of film genre
queryString = """
SELECT DISTINCT ?o ?oName
WHERE {
     ?o wdt:P31 wd:Q201658.
   ?o sc:name ?oName.
}
ORDER BY ASC(?oName)
LIMIT 1000
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q4674071'), ('oName', 'Acid Western')]
[('o', 'http://www.wikidata.org/entity/Q108179271'), ('oName', 'Arthouse animation')]
[('o', 'http://www.wikidata.org/entity/Q223770'), ('oName', 'B movie')]
[('o', 'http://www.wikidata.org/entity/Q4836991'), ('oName', 'B western')]
[('o', 'http://www.wikidata.org/entity/Q3461143'), ('oName', 'BDSM film')]
[('o', 'http://www.wikidata.org/entity/Q31086090'), ('oName', 'Barbarian film')]
[('o', 'http://www.wikidata.org/entity/Q84041'), ('oName', 'Bavarian porn')]
[('o', 'http://www.wikidata.org/entity/Q7717463'), ('oName', 'Bible film')]
[('o', 'http://www.wikidata.org/entity/Q48816645'), ('oName', 'Black film')]
[('o', 'http://www.wikidata.org/entity/Q93196'), ('oName', 'Bollywood')]
[('o', 'http://www.wikidata.org/entity/Q16247268'), ('oName', 'Bollywood horror film')]
[('o', 'http://www.wikidata.org/entity/Q4950139'), ('oName', 'Bourekas film')]
[('o', 'http://www.wikidata.org/entity/Q849666'), ('oNa

In [24]:
#film genre instance of
queryString = """
SELECT DISTINCT ?o ?oName
WHERE {
     wd:Q201658 wdt:P31 ?o.
   ?o sc:name ?oName.
}
ORDER BY ASC(?oName)
LIMIT 1000
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q19361238'), ('oName', 'Wikidata metaclass')]
1


In [25]:
#prova finale
queryString = """
SELECT DISTINCT ?comp ?name (COUNT(?film) AS ?NcrimeFilm)
WHERE {
    ?film wdt:P136 wd:Q959790; #genre crime film
        wdt:P272 ?comp.        #produced by comp
    VALUES ?prodComp {wd:Q10689397 wd:Q1762059} #either tv or film (resp.) prod comp
    ?comp wdt:P31 ?prodComp;
        sc:name ?name.
}
GROUP BY ?comp ?name
ORDER BY DESC(?NcrimeFilm)
LIMIT 5
"""

print("Results")
x=run_query(queryString)

Results
[('comp', 'http://www.wikidata.org/entity/Q434841'), ('name', '20th Century Studios'), ('NcrimeFilm', '180')]
[('comp', 'http://www.wikidata.org/entity/Q126399'), ('name', 'Warner Bros.'), ('NcrimeFilm', '164')]
[('comp', 'http://www.wikidata.org/entity/Q186941'), ('name', 'Columbia Pictures'), ('NcrimeFilm', '118')]
[('comp', 'http://www.wikidata.org/entity/Q168383'), ('name', 'Universal Pictures'), ('NcrimeFilm', '113')]
[('comp', 'http://www.wikidata.org/entity/Q179200'), ('name', 'Metro-Goldwyn-Mayer'), ('NcrimeFilm', '112')]
5


Final query for this task

In [26]:
#top 5 crime film producers
queryString = """
SELECT DISTINCT ?iri ?name (COUNT(?film) AS ?NcrimeFilm)
WHERE {
    ?film wdt:P136 wd:Q959790; #genre crime film
        wdt:P272 ?iri.        #produced by comp (iri)
    VALUES ?prodComp {wd:Q10689397 wd:Q1762059} #either tv or film (resp.) prod comp
    ?iri wdt:P31 ?prodComp;
        sc:name ?name.
}
GROUP BY ?iri ?name
ORDER BY DESC(?NcrimeFilm)
LIMIT 5
"""

print("Results")
x=run_query(queryString)

Results
[('iri', 'http://www.wikidata.org/entity/Q434841'), ('name', '20th Century Studios'), ('NcrimeFilm', '180')]
[('iri', 'http://www.wikidata.org/entity/Q126399'), ('name', 'Warner Bros.'), ('NcrimeFilm', '164')]
[('iri', 'http://www.wikidata.org/entity/Q186941'), ('name', 'Columbia Pictures'), ('NcrimeFilm', '118')]
[('iri', 'http://www.wikidata.org/entity/Q168383'), ('name', 'Universal Pictures'), ('NcrimeFilm', '113')]
[('iri', 'http://www.wikidata.org/entity/Q179200'), ('name', 'Metro-Goldwyn-Mayer'), ('NcrimeFilm', '112')]
5


## Task 6
Find the company (amongst television production company and film production company) which distributed more films (of any kind of film genre) that they did not produce (the result set must be a list of triples company IRI, label and #film).

In [27]:
#prova finale
queryString = """
SELECT DISTINCT (?comp AS ?iri) ?name (COUNT(?film) AS ?Nfilm)
WHERE {    
    VALUES ?prodComp {wd:Q10689397 wd:Q1762059} #either tv or film (resp.) prod comp
    ?comp wdt:P31 ?prodComp;
        sc:name ?name.
    ?film wdt:P750 ?comp.                     #distributed by comp
    FILTER NOT EXISTS { ?film wdt:P272 ?comp} #not produced by comp
}
GROUP BY ?comp ?name
ORDER BY DESC(?Nfilm)
LIMIT 1
"""

print("Results")
x=run_query(queryString)

Results
[('iri', 'http://www.wikidata.org/entity/Q907311'), ('name', 'Netflix'), ('Nfilm', '26171')]
1


Final query for this task

In [28]:
#company with most distributed films not produced by them
queryString = """
SELECT DISTINCT (?comp AS ?iri) ?name (COUNT(?film) AS ?Nfilm)
WHERE {    
    VALUES ?prodComp {wd:Q10689397 wd:Q1762059} #either tv or film (resp.) prod comp
    ?comp wdt:P31 ?prodComp;
        sc:name ?name.
    ?film wdt:P750 ?comp.                     #distributed by comp
    FILTER NOT EXISTS { ?film wdt:P272 ?comp} #not produced by comp
}
GROUP BY ?comp ?name
ORDER BY DESC(?Nfilm)
LIMIT 1
"""

print("Results")
x=run_query(queryString)

Results
[('iri', 'http://www.wikidata.org/entity/Q907311'), ('name', 'Netflix'), ('Nfilm', '26171')]
1


## Task 7
Find how many company are listed in every stock exchange market (the result set must be a list of triples stock exchange IRI, label and #companies).

In [29]:
#all stock exchange markets
queryString = """
SELECT DISTINCT ?o ?name
WHERE {    
    ?comp wdt:P414 ?o.
    ?o sc:name ?name.
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1526647'), ('name', 'Philippine Stock Exchange')]
[('o', 'http://www.wikidata.org/entity/Q43335'), ('name', 'Bursa Malaysia')]
[('o', 'http://www.wikidata.org/entity/Q2415561'), ('name', 'Euronext Lisbon')]
[('o', 'http://www.wikidata.org/entity/Q891561'), ('name', 'Santiago Stock Exchange')]
[('o', 'http://www.wikidata.org/entity/Q550050'), ('name', 'Belgrade Stock Exchange')]
[('o', 'http://www.wikidata.org/entity/Q59551'), ('name', 'Warsaw Stock Exchange')]
[('o', 'http://www.wikidata.org/entity/Q74662'), ('name', 'Fundsquare Market Infrastructure')]
[('o', 'http://www.wikidata.org/entity/Q851259'), ('name', 'Budapest Stock Exchange')]
[('o', 'http://www.wikidata.org/entity/Q698535'), ('name', 'Wiener Börse')]
[('o', 'http://www.wikidata.org/entity/Q3349831'), ('name', 'Chittagong Stock Exchange')]
[('o', 'http://www.wikidata.org/entity/Q66320'), ('name', 'over-the-counter trading')]
[('o', 'http://www.wikidata.org/entity/Q491503'), ('

In [30]:
#prova finale
queryString = """
SELECT DISTINCT (?sem AS ?iri) ?name (COUNT(?comp) AS ?Ncompanies)
WHERE {    
    VALUES ?prodComp {wd:Q10689397 wd:Q1762059} #either tv or film (resp.) prod comp
    ?comp wdt:P31 ?prodComp;
        wdt:P414 ?sem. #sem is stock exchange market
    ?sem sc:name ?name.
}
GROUP BY ?sem ?name
ORDER BY DESC(?Ncompanies)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('iri', 'http://www.wikidata.org/entity/Q13677'), ('name', 'New York Stock Exchange'), ('Ncompanies', '5')]
[('iri', 'http://www.wikidata.org/entity/Q82059'), ('name', 'NASDAQ'), ('Ncompanies', '4')]
[('iri', 'http://www.wikidata.org/entity/Q1661737'), ('name', 'Indonesia Stock Exchange'), ('Ncompanies', '2')]
[('iri', 'http://www.wikidata.org/entity/Q496672'), ('name', 'Hong Kong Stock Exchange'), ('Ncompanies', '2')]
[('iri', 'http://www.wikidata.org/entity/Q217475'), ('name', 'Tokyo Stock Exchange'), ('Ncompanies', '2')]
[('iri', 'http://www.wikidata.org/entity/Q732670'), ('name', 'Australian Securities Exchange'), ('Ncompanies', '1')]
[('iri', 'http://www.wikidata.org/entity/Q818723'), ('name', 'Toronto Stock Exchange'), ('Ncompanies', '1')]
7


In [31]:
#all tv or film prod companies
queryString = """
SELECT DISTINCT ?comp ?name 
WHERE {    
    VALUES ?prodComp {wd:Q10689397 wd:Q1762059} #either tv or film (resp.) prod comp
    ?comp wdt:P31 ?prodComp;        
        sc:name ?name.
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('comp', 'http://www.wikidata.org/entity/Q1124579'), ('name', 'Condor Films')]
[('comp', 'http://www.wikidata.org/entity/Q20724376'), ('name', 'CC&C – Clarke Costelle et Cie')]
[('comp', 'http://www.wikidata.org/entity/Q17086011'), ('name', 'Prometheus Entertainment')]
[('comp', 'http://www.wikidata.org/entity/Q17088846'), ('name', 'Spring Films')]
[('comp', 'http://www.wikidata.org/entity/Q17090410'), ('name', 'Tribe Pictures')]
[('comp', 'http://www.wikidata.org/entity/Q17090464'), ('name', 'Trump Productions')]
[('comp', 'http://www.wikidata.org/entity/Q17090873'), ('name', 'Mosdeux')]
[('comp', 'http://www.wikidata.org/entity/Q18155354'), ('name', 'TSG Entertainment')]
[('comp', 'http://www.wikidata.org/entity/Q477943'), ('name', 'Bad Hat Harry Productions')]
[('comp', 'http://www.wikidata.org/entity/Q5433853'), ('name', 'Fangoria Films')]
[('comp', 'http://www.wikidata.org/entity/Q7548638'), ('name', 'Snowfort Pictures')]
[('comp', 'http://www.wikidata.org/entity/Q1720138

In [32]:
#all tv or film prod companies that have stock exchange market
queryString = """
SELECT DISTINCT ?comp ?name 
WHERE {    
    VALUES ?prodComp {wd:Q10689397 wd:Q1762059} #either tv or film (resp.) prod comp
    ?comp wdt:P31 ?prodComp;        
        sc:name ?name.
    FILTER EXISTS {?comp wdt:P414 ?o}
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('comp', 'http://www.wikidata.org/entity/Q541569'), ('name', 'Orange Sky Golden Harvest')]
[('comp', 'http://www.wikidata.org/entity/Q500088'), ('name', 'DreamWorks Animation')]
[('comp', 'http://www.wikidata.org/entity/Q7414'), ('name', 'The Walt Disney Company')]
[('comp', 'http://www.wikidata.org/entity/Q907311'), ('name', 'Netflix')]
[('comp', 'http://www.wikidata.org/entity/Q2140077'), ('name', 'Tribune Media')]
[('comp', 'http://www.wikidata.org/entity/Q5395348'), ('name', 'Eros International')]
[('comp', 'http://www.wikidata.org/entity/Q1654103'), ('name', 'IMAX Corporation')]
[('comp', 'http://www.wikidata.org/entity/Q604775'), ('name', '4Kids Entertainment')]
[('comp', 'http://www.wikidata.org/entity/Q3299128'), ('name', 'Rainmaker Entertainment')]
[('comp', 'http://www.wikidata.org/entity/Q1476080'), ('name', 'Private Media Group')]
[('comp', 'http://www.wikidata.org/entity/Q56085408'), ('name', 'MD Pictures')]
[('comp', 'http://www.wikidata.org/entity/Q825291'), ('n

In [33]:
#names of companies per stock exchange market
queryString = """
SELECT ?name ?cName
WHERE {    
    VALUES ?prodComp {wd:Q10689397 wd:Q1762059} #either tv or film (resp.) prod comp
    ?comp wdt:P31 ?prodComp;
        wdt:P414 ?sem; #sem is stock exchange market
        sc:name ?cName.
    ?sem sc:name ?name.
}
GROUP BY ?sem ?name
ORDER BY ASC(?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'Australian Securities Exchange'), ('cName', 'Village Roadshow')]
[('name', 'Hong Kong Stock Exchange'), ('cName', 'Orange Sky Golden Harvest')]
[('name', 'Hong Kong Stock Exchange'), ('cName', 'Alibaba Pictures')]
[('name', 'Indonesia Stock Exchange'), ('cName', 'MD Pictures')]
[('name', 'NASDAQ'), ('cName', 'Private Media Group')]
[('name', 'NASDAQ'), ('cName', 'DreamWorks Animation')]
[('name', 'NASDAQ'), ('cName', 'Netflix')]
[('name', 'New York Stock Exchange'), ('cName', 'IMAX Corporation')]
[('name', 'New York Stock Exchange'), ('cName', 'Tribune Media')]
[('name', 'New York Stock Exchange'), ('cName', 'Eros International')]
[('name', 'New York Stock Exchange'), ('cName', '4Kids Entertainment')]
[('name', 'New York Stock Exchange'), ('cName', 'The Walt Disney Company')]
[('name', 'Tokyo Stock Exchange'), ('cName', 'The Walt Disney Company')]
[('name', 'Tokyo Stock Exchange'), ('cName', 'Tohokushinsha Film')]
[('name', 'Toronto Stock Exchange'), ('cName', 'Rainm

Final query for this task

In [34]:
#number of film/tv companies per stock exchange market 
#I included this final query because it was what i expected to be asked in the task, 
#the final query satisfying the task for all kind of companies is in the cell below
queryString = """
SELECT (?sem AS ?iri) ?name (COUNT(DISTINCT ?comp) AS ?Ncompanies)
WHERE {    
    VALUES ?prodComp {wd:Q10689397 wd:Q1762059} #either tv or film (resp.) prod comp
    ?comp wdt:P31 ?prodComp;
        wdt:P414 ?sem. #sem is stock exchange market
    ?sem sc:name ?name.
}
GROUP BY ?sem ?name
ORDER BY DESC(?Ncompanies) ASC(?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('iri', 'http://www.wikidata.org/entity/Q13677'), ('name', 'New York Stock Exchange'), ('Ncompanies', '5')]
[('iri', 'http://www.wikidata.org/entity/Q82059'), ('name', 'NASDAQ'), ('Ncompanies', '3')]
[('iri', 'http://www.wikidata.org/entity/Q496672'), ('name', 'Hong Kong Stock Exchange'), ('Ncompanies', '2')]
[('iri', 'http://www.wikidata.org/entity/Q217475'), ('name', 'Tokyo Stock Exchange'), ('Ncompanies', '2')]
[('iri', 'http://www.wikidata.org/entity/Q732670'), ('name', 'Australian Securities Exchange'), ('Ncompanies', '1')]
[('iri', 'http://www.wikidata.org/entity/Q1661737'), ('name', 'Indonesia Stock Exchange'), ('Ncompanies', '1')]
[('iri', 'http://www.wikidata.org/entity/Q818723'), ('name', 'Toronto Stock Exchange'), ('Ncompanies', '1')]
7


In [35]:
#number of all kind of companies per stock exchange market
queryString = """
SELECT (?sem AS ?iri) ?name (COUNT(DISTINCT ?comp) AS ?Ncompanies)
WHERE {    
    
    ?comp wdt:P414 ?sem. #sem is stock exchange market
    ?sem sc:name ?name.
}
GROUP BY ?sem ?name
ORDER BY DESC(?Ncompanies) ASC(?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('iri', 'http://www.wikidata.org/entity/Q13677'), ('name', 'New York Stock Exchange'), ('Ncompanies', '1832')]
[('iri', 'http://www.wikidata.org/entity/Q496672'), ('name', 'Hong Kong Stock Exchange'), ('Ncompanies', '1799')]
[('iri', 'http://www.wikidata.org/entity/Q82059'), ('name', 'NASDAQ'), ('Ncompanies', '1755')]
[('iri', 'http://www.wikidata.org/entity/Q1661737'), ('name', 'Indonesia Stock Exchange'), ('Ncompanies', '697')]
[('iri', 'http://www.wikidata.org/entity/Q171240'), ('name', 'London Stock Exchange'), ('Ncompanies', '604')]
[('iri', 'http://www.wikidata.org/entity/Q517750'), ('name', 'Shenzhen Stock Exchange'), ('Ncompanies', '493')]
[('iri', 'http://www.wikidata.org/entity/Q217475'), ('name', 'Tokyo Stock Exchange'), ('Ncompanies', '460')]
[('iri', 'http://www.wikidata.org/entity/Q495364'), ('name', 'Korea Stock Exchange'), ('Ncompanies', '417')]
[('iri', 'http://www.wikidata.org/entity/Q818723'), ('name', 'Toronto Stock Exchange'), ('Ncompanies', '394')]
[('iri

## Task 8
Identify the BGP for Academy Award

In [36]:
# Lion King awards
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q36479 wdt:P166 ?obj .
   # get the label
   ?obj sc:name ?name.
}
ORDER BY ASC(?name)
LIMIT 1000
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q488651'), ('name', 'Academy Award for Best Original Score')]
[('obj', 'http://www.wikidata.org/entity/Q823422'), ('name', 'National Film Registry')]
2


In [37]:
# AA for Best original score instance of
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q488651 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
ORDER BY ASC(?name)
LIMIT 1000
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q19020'), ('name', 'Academy Awards')]
[('obj', 'http://www.wikidata.org/entity/Q96474685'), ('name', 'award for best original music')]
2


In [38]:
# AAs all props
queryString = """
SELECT DISTINCT ?p ?pName ?name
WHERE {
   # bind something
   wd:Q19020 ?p ?obj .
   # get the label
   ?p sc:name ?pName.
   ?obj sc:name ?name.
}
ORDER BY ASC(?name)
LIMIT 1000
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('name', 'Academy Award')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('name', 'Academy Award for Best Actor')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('name', 'Academy Award for Best Actress')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('name', 'Academy Award for Best Animated Feature')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('name', 'Academy Award for Best Cinematography')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('name', 'Academy Award for Best Costume Design')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('name', 'Academy Award for Best Director')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('name', 'Academy Award for Best Director (Comedy picture)')]
[('p', 'ht

In [39]:
# AAs different from
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q19020 wdt:P1889 ?obj .
   # get the label
   ?obj sc:name ?name.
}
ORDER BY ASC(?name)
LIMIT 1000
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q4671334'), ('name', 'Academy Award')]
[('obj', 'http://www.wikidata.org/entity/Q16913666'), ('name', 'Academy Awards ceremony')]
[('obj', 'http://www.wikidata.org/entity/Q21475960'), ('name', 'Óscar')]
3


Final query for this task

In [40]:
# BGP for Academy Award
queryString = """
SELECT *
WHERE {
   BIND(wd:Q4671334 AS ?iri)
   ?iri sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('iri', 'http://www.wikidata.org/entity/Q4671334'), ('name', 'Academy Award')]
1


## Task 9
Find the companies (among television and film production companies) that won at least 5 Academy Awards for Best Actress for the movies they produced (the result set must be a list of triples company IRI, label and #awards).

In [41]:
# AAs all parts
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q19020 wdt:P527 ?obj .
   # get the label
   ?obj sc:name ?name.
}
ORDER BY ASC(?name)
LIMIT 1000
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q103916'), ('name', 'Academy Award for Best Actor')]
[('obj', 'http://www.wikidata.org/entity/Q103618'), ('name', 'Academy Award for Best Actress')]
[('obj', 'http://www.wikidata.org/entity/Q106800'), ('name', 'Academy Award for Best Animated Feature')]
[('obj', 'http://www.wikidata.org/entity/Q131520'), ('name', 'Academy Award for Best Cinematography')]
[('obj', 'http://www.wikidata.org/entity/Q277536'), ('name', 'Academy Award for Best Costume Design')]
[('obj', 'http://www.wikidata.org/entity/Q103360'), ('name', 'Academy Award for Best Director')]
[('obj', 'http://www.wikidata.org/entity/Q3451157'), ('name', 'Academy Award for Best Director (Comedy picture)')]
[('obj', 'http://www.wikidata.org/entity/Q111332'), ('name', 'Academy Award for Best Documentary Feature')]
[('obj', 'http://www.wikidata.org/entity/Q105304'), ('name', 'Academy Award for Best Foreign Language Film')]
[('obj', 'http://www.wikidata.org/entity/Q22235329'), ('name'

In [42]:
#prova finale
queryString = """
SELECT DISTINCT (?comp AS ?iri) ?name (COUNT(?film) AS ?Nawards)
WHERE {
    VALUES ?prodComp {wd:Q10689397 wd:Q1762059} #either tv or film (resp.) prod comp
    ?comp wdt:P31 ?prodComp;
        sc:name ?name.
    ?film wdt:P272 ?comp;                       #film produced by comp
        wdt:P166 wd:Q103618.                    #     won best actress

}
GROUP BY ?comp ?name
HAVING(COUNT(?film) > 4)
ORDER BY DESC(?Nawards)                        #VALUES PUO'PRENDERE DOPPI SE VALE PER ENTRAMBI 
LIMIT 100                                      #(sia tv che film in questo caso)

"""

print("Results")
x=run_query(queryString)

Results
[('iri', 'http://www.wikidata.org/entity/Q434841'), ('name', '20th Century Studios'), ('Nawards', '14')]
[('iri', 'http://www.wikidata.org/entity/Q179200'), ('name', 'Metro-Goldwyn-Mayer'), ('Nawards', '11')]
[('iri', 'http://www.wikidata.org/entity/Q159846'), ('name', 'Paramount Pictures'), ('Nawards', '11')]
[('iri', 'http://www.wikidata.org/entity/Q126399'), ('name', 'Warner Bros.'), ('Nawards', '9')]
4


In [43]:
#all best actress oscar films of 20th Cent.
queryString = """
SELECT ?name 
WHERE {
    ?film wdt:P272 wd:Q434841;                       #film produced by 20th Cent
        wdt:P166 wd:Q103618;                         #won best actress
        sc:name ?name.                    
}

ORDER BY DESC(?Nawards)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('name', 'Seventh Heaven')]
[('name', 'Walk the Line')]
[('name', 'Sunrise: A Song of Two Humans')]
[('name', 'The Three Faces of Eve')]
[('name', 'Anastasia')]
[('name', 'The Prime of Miss Jean Brodie')]
[('name', 'The Song of Bernadette')]
7


In [44]:
#prova finale 2
queryString = """
SELECT DISTINCT (?comp AS ?iri) ?name (COUNT(DISTINCT ?film) AS ?Nawards)
WHERE {
    VALUES ?prodComp {wd:Q10689397 wd:Q1762059} #either tv or film (resp.) prod comp
    ?comp wdt:P31 ?prodComp;
        sc:name ?name.
    ?film wdt:P272 ?comp;                       #film produced by comp
        wdt:P166 wd:Q103618.                    #     won best actress

}
GROUP BY ?comp ?name
HAVING(COUNT(DISTINCT ?film) > 4)
ORDER BY DESC(?Nawards) ASC(?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('iri', 'http://www.wikidata.org/entity/Q179200'), ('name', 'Metro-Goldwyn-Mayer'), ('Nawards', '11')]
[('iri', 'http://www.wikidata.org/entity/Q159846'), ('name', 'Paramount Pictures'), ('Nawards', '11')]
[('iri', 'http://www.wikidata.org/entity/Q126399'), ('name', 'Warner Bros.'), ('Nawards', '9')]
[('iri', 'http://www.wikidata.org/entity/Q434841'), ('name', '20th Century Studios'), ('Nawards', '7')]
4


Final query for this task

In [45]:
#number of best actress award winning films produced by each company
queryString = """
SELECT DISTINCT (?comp AS ?iri) ?name (COUNT(DISTINCT ?film) AS ?Nawards)
WHERE {
    VALUES ?prodComp {wd:Q10689397 wd:Q1762059} #either tv or film (resp.) prod comp
    ?comp wdt:P31 ?prodComp;
        sc:name ?name.
    ?film wdt:P272 ?comp;                       #film produced by comp
        wdt:P166 wd:Q103618.                    #     won best actress

}
GROUP BY ?comp ?name
HAVING(COUNT(DISTINCT ?film) > 4)
ORDER BY DESC(?Nawards) ASC(?name)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('iri', 'http://www.wikidata.org/entity/Q179200'), ('name', 'Metro-Goldwyn-Mayer'), ('Nawards', '11')]
[('iri', 'http://www.wikidata.org/entity/Q159846'), ('name', 'Paramount Pictures'), ('Nawards', '11')]
[('iri', 'http://www.wikidata.org/entity/Q126399'), ('name', 'Warner Bros.'), ('Nawards', '9')]
[('iri', 'http://www.wikidata.org/entity/Q434841'), ('name', '20th Century Studios'), ('Nawards', '7')]
4
